# **Setting up the environment**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
from sklearn.datasets import load_diabetes

data = load_diabetes()

In [4]:
data.data

array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
         0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
        -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
         0.00286131, -0.02593034],
       ...,
       [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
        -0.04688253,  0.01549073],
       [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
         0.04452873, -0.02593034],
       [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
        -0.00422151,  0.00306441]])

In [5]:
len(data.data[0])

10

In [6]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(
    data.data, data.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

# **Plain vanilla DNN**

In [7]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 10)               21        
 n)                                                              
                                                                 
 dense (Dense)               (None, 50)                550       
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 5,722
Trainable params: 5,701
Non-trainable params: 21
_____________________________________________________

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test, rmse_test = model.evaluate(X_test, y_test)

Epoch 1/20
8/8 [==============================] - 6s 62ms/step - loss: 28757.0430 - root_mean_squared_error: 169.5790 - val_loss: 32466.0156 - val_root_mean_squared_error: 180.1833
Epoch 2/20
8/8 [==============================] - 0s 8ms/step - loss: 28504.0645 - root_mean_squared_error: 168.8315 - val_loss: 32159.7891 - val_root_mean_squared_error: 179.3315
Epoch 3/20
8/8 [==============================] - 0s 9ms/step - loss: 28175.7090 - root_mean_squared_error: 167.8562 - val_loss: 31735.1992 - val_root_mean_squared_error: 178.1438
Epoch 4/20
8/8 [==============================] - 0s 9ms/step - loss: 27684.6445 - root_mean_squared_error: 166.3870 - val_loss: 31077.9102 - val_root_mean_squared_error: 176.2893
Epoch 5/20
8/8 [==============================] - 0s 8ms/step - loss: 26951.1875 - root_mean_squared_error: 164.1682 - val_loss: 30035.9707 - val_root_mean_squared_error: 173.3089
Epoch 6/20
8/8 [==============================] - 0s 10ms/step - loss: 25793.7637 - root_mean_squar

# **Wide & Deep Neural Networks**

In [10]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [11]:
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(50, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(50, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

model = tf.keras.Model(inputs=[input_], outputs=[output])

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 normalization (Normalization)  (None, 10)           21          ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 50)           550         ['normalization[0][0]']          
                                                                                                  
 dense_1 (Dense)                (None, 50)           2550        ['dense[0][0]']                  
                                                                                              

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
normalization_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)

Epoch 1/20
8/8 [==============================] - 1s 27ms/step - loss: 28708.3496 - root_mean_squared_error: 169.4354 - val_loss: 32493.6445 - val_root_mean_squared_error: 180.2599
Epoch 2/20
8/8 [==============================] - 0s 9ms/step - loss: 28554.8262 - root_mean_squared_error: 168.9817 - val_loss: 32310.9883 - val_root_mean_squared_error: 179.7526
Epoch 3/20
8/8 [==============================] - 0s 8ms/step - loss: 28375.1543 - root_mean_squared_error: 168.4493 - val_loss: 32095.4160 - val_root_mean_squared_error: 179.1519
Epoch 4/20
8/8 [==============================] - 0s 9ms/step - loss: 28157.7832 - root_mean_squared_error: 167.8028 - val_loss: 31827.1504 - val_root_mean_squared_error: 178.4017
Epoch 5/20
8/8 [==============================] - 0s 7ms/step - loss: 27894.8770 - root_mean_squared_error: 167.0176 - val_loss: 31480.1660 - val_root_mean_squared_error: 177.4265
Epoch 6/20
8/8 [==============================] - 0s 8ms/step - loss: 27547.0918 - root_mean_square

In [14]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [15]:
input_wide = tf.keras.layers.Input(shape=[5])  # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[8])  # features 2 to 9
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(50, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(50, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 normalization_1 (Normalization  (None, 8)           17          ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 input_1 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 50)           450         ['normalization_1[0][0]']    

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20,
                    validation_data=((X_valid_wide, X_valid_deep), y_valid))
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)

Epoch 1/20
8/8 [==============================] - 1s 29ms/step - loss: 28845.7539 - root_mean_squared_error: 169.8404 - val_loss: 32624.3340 - val_root_mean_squared_error: 180.6221
Epoch 2/20
8/8 [==============================] - 0s 8ms/step - loss: 28707.6445 - root_mean_squared_error: 169.4333 - val_loss: 32459.8555 - val_root_mean_squared_error: 180.1662
Epoch 3/20
8/8 [==============================] - 0s 8ms/step - loss: 28548.6484 - root_mean_squared_error: 168.9635 - val_loss: 32268.1270 - val_root_mean_squared_error: 179.6333
Epoch 4/20
8/8 [==============================] - 0s 10ms/step - loss: 28351.8926 - root_mean_squared_error: 168.3802 - val_loss: 32032.9961 - val_root_mean_squared_error: 178.9776
Epoch 5/20
8/8 [==============================] - 0s 9ms/step - loss: 28122.9395 - root_mean_squared_error: 167.6990 - val_loss: 31732.9375 - val_root_mean_squared_error: 178.1374
Epoch 6/20
8/8 [==============================] - 0s 8ms/step - loss: 27822.5605 - root_mean_squar

In [18]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [19]:
input_wide = tf.keras.layers.Input(shape=[5])  # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[8])  # features 2 to 9
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(50, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(50, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)
model = tf.keras.Model(inputs=[input_wide, input_deep],
                       outputs=[output, aux_output])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 normalization_1 (Normalization  (None, 8)           17          ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 input_1 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 50)           450         ['normalization_1[0][0]']    

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

In [22]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
8/8 [==============================] - 2s 41ms/step - loss: 28799.0332 - dense_2_loss: 28798.6230 - dense_3_loss: 28802.7031 - dense_2_root_mean_squared_error: 169.7016 - dense_3_root_mean_squared_error: 169.7136 - val_loss: 32554.0547 - val_dense_2_loss: 32544.6777 - val_dense_3_loss: 32638.4180 - val_dense_2_root_mean_squared_error: 180.4014 - val_dense_3_root_mean_squared_error: 180.6611
Epoch 2/20
8/8 [==============================] - 0s 12ms/step - loss: 28667.1973 - dense_2_loss: 28656.7656 - dense_3_loss: 28761.1230 - dense_2_root_mean_squared_error: 169.2831 - dense_3_root_mean_squared_error: 169.5910 - val_loss: 32397.5234 - val_dense_2_loss: 32376.3184 - val_dense_3_loss: 32588.3770 - val_dense_2_root_mean_squared_error: 179.9342 - val_dense_3_root_mean_squared_error: 180.5225
Epoch 3/20
8/8 [==============================] - 0s 10ms/step - loss: 28516.8770 - dense_2_loss: 28495.6328 - dense_3_loss: 28708.0957 - dense_2_root_mean_squared_error: 168.8065 - dense_3_

In [23]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

4/4 [==============================] - 0s 5ms/step - loss: 11190.7070 - dense_2_loss: 10799.4258 - dense_3_loss: 14712.2490 - dense_2_root_mean_squared_error: 103.9203 - dense_3_root_mean_squared_error: 121.2941


In [24]:
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)  # needed to support naming the model
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output


In [25]:
tf.random.set_seed(42)
model = WideAndDeepModel(50, activation="relu", name="model")

In [26]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=optimizer,
              metrics=["RootMeanSquaredError"])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)))
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

Epoch 1/20
8/8 [==============================] - 2s 42ms/step - loss: 28799.7656 - output_1_loss: 28792.8691 - output_2_loss: 28861.8223 - output_1_root_mean_squared_error: 169.6846 - output_2_root_mean_squared_error: 169.8877 - val_loss: 32570.8711 - val_output_1_loss: 32559.1445 - val_output_2_loss: 32676.4043 - val_output_1_root_mean_squared_error: 180.4415 - val_output_2_root_mean_squared_error: 180.7662
Epoch 2/20
8/8 [==============================] - 0s 10ms/step - loss: 28662.1699 - output_1_loss: 28647.4980 - output_2_loss: 28794.1816 - output_1_root_mean_squared_error: 169.2557 - output_2_root_mean_squared_error: 169.6885 - val_loss: 32400.6934 - val_output_1_loss: 32379.3906 - val_output_2_loss: 32592.3770 - val_output_1_root_mean_squared_error: 179.9427 - val_output_2_root_mean_squared_error: 180.5336
Epoch 3/20
8/8 [==============================] - 0s 11ms/step - loss: 28501.1094 - output_1_loss: 28477.5117 - output_2_loss: 28713.5117 - output_1_root_mean_squared_error: 